In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score
import xgboost as xgb

In [2]:
df = pd.read_csv("CKD.csv",index_col=None)

In [3]:
df = pd.get_dummies(df, drop_first=True)

In [4]:
df

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [5]:
X = df.drop("classification_yes", axis=1)
y = df["classification_yes"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [6]:

rf = RandomForestClassifier(n_estimators=10,criterion='entropy',random_state=0)
rf.fit(X_train, y_train)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)

C:\Anaconda\envs\mlai\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [48]:

rf_importance = pd.DataFrame({
    'Feature': X.columns,
    'RandomForest': rf.feature_importances_
     
})

# Logistic Regression (absolute coefficients)
logreg_importance = pd.DataFrame({
    'Feature': X.columns,
    'LogisticRegression': np.abs(logreg.coef_[0])
    
})

# Permutation importance (using RandomForest)
perm = permutation_importance(rf, X_test, y_test, n_repeats=10, random_state=42)
perm_importance = pd.DataFrame({
    'Feature': X.columns,
    'PermutationImportance': perm.importances_mean
    
})

# XGBoost importance
xgb_importance = pd.DataFrame({
    'Feature': X.columns,
    'XGBoost': xgb_model.feature_importances_
    
})


In [49]:
all_importances = pd.concat([rf_importance, logreg_importance, perm_importance, xgb_importance], axis=1)

In [50]:
all_importances

,Feature,RandomForest,Feature,LogisticRegression,Feature,PermutationImportance,Feature,XGBoost
0,age,0.020179,age,0.022255,age,-0.006,age,0.008251
1,bp,0.004212,bp,0.076803,bp,-0.002,bp,0.005452
2,al,0.071986,al,0.130235,al,0.002,al,0.123546
3,su,0.007468,su,0.009083,su,-0.003,su,0.000000
4,bgr,0.033663,bgr,0.043034,bgr,-0.003,bgr,0.047036
5,bu,0.048943,bu,0.010241,bu,-0.002,bu,0.005326
6,sc,0.045899,sc,0.127960,sc,-0.007,sc,0.010826
7,sod,0.032434,sod,0.080452,sod,0.000,sod,0.003207
8,pot,0.013443,pot,0.001331,pot,0.000,pot,0.004116
9,hrmo,0.201079,hrmo,0.213003,hrmo,0.026,hrmo,0.323080
